Bitcoin에 대한 구조를 알아보기 위해 일단 Bitcoin을 설치해보고 간단하게 거래 테스트를 해보았습니다.

<!-- TEASER_END -->

# Bitcoin 설치

운영체제: Ubuntu 16.04 LTS

Github 에서 bitcoin 소스를 다운로드 받습니다.

```
git clone https://github.com/bitcoin/bitcoin.git
```

bitcoin 의존성 패키지 목록 소스의 docs/dependencies.md 파일 또는 아래 링크를 참조하세요

https://github.com/bitcoin/bitcoin/blob/master/doc/dependencies.md

의존성 패키지 설치를 설치합니다.

gcc 빌드 툴 및 python3

```
sudo apt-get update
sudo apt-get install -y build-essential bsdmainutils autoconf automake autotools-dev pkg-config libtool python3 
```

openssl, boost, event, miniupnpc, zmq 라이브러리

```
sudo apt-get install -y libssl-dev libboost-all-dev libevent-dev libminiupnpc-dev libzmq3-dev
```

db4.8 (Ubuntu는 기본 5.1 버전을 사용하기 때문에 4.8 버전으로 다시 설치)

```
sudo apt-get install -y software-properties-common
sudo add-apt-repository ppa:bitcoin/bitcoin
sudo apt-get update
sudo apt-get install -y libdb4.8-dev libdb4.8++-dev
```

GUI 를 위한 QT5 설치

```
sudo apt-get install -y libqt5gui5 libqt5core5a libqt5dbus5 qttools5-dev qttools5-dev-tools libprotobuf-dev protobuf-compiler
sudo apt-get install -y libqrencode-dev
```

bitcoin 컴파일

```
cd bitcoin
./autogen.sh
./configure
make
```

문제 없이 컴파일이 완료되었으면 아래 명령으로 설치를 완료합니다.

```
sudo make install
```


# Bitcoin Core 실행

비트코인을 실제 네트워크 상에서 돌리기보다는 동작 방식을 익히기 위해 테스트로 사용합니다.

* Testnet: 인터넷 상의 테스트 네트워크
* Regtest: 로컬 네트워크상의 테스트 네트워크

Testnet 은 인터넷 상에 공개된 테스트 네트워크여서 이미 대량의 블록 체인이 존재하기 때문에 초기 기동시 모든 데이터를 다운로드 받는데 시간이 걸리므로 간단한 테스트용으로는 Regtest 가 적당합니다. 

게다가 Bitcoin Core 에서는 0.13 버전 이후로 테스트넷 이외에서 마이닝 하는 기능을 빼버렸기 때문에 굳이 모든 데이터를 다운로드 받을 필요가 없습니다.

사용을 위한 모든 정보는 틀별히 설정하지 않았다면 계정의 .bitcoin 디렉안에 있습니다.

아래와 같이 daemon 형식으로 띄워 줍니다.

```
bitcoind -regtest -daemon
```


# Bitcoin 거래 테스트

송금 테스트를 하기 위해서 테스트용 비트코인을 생성해 봅니다. 블록을 생성(채굴)한 보상으로 비트코인을 받을 수 있으며, 채굴자는 보상 후 100블록 이후에 송금이 가능하기 때문에 101개의 블록을 생성해 봅니다.

```
bitcoin-cli -regtest generate 101
[
  "05f8a8ea539fc9d51538e45dc501543da9ead2b7321c6224bfff053b74247ef8",
  "4a2a7cdd7fa6948f33b1507de7ba03501b9f352833d3690c54f95e1a5b4c1b75",
  ...
  "6a5af93afbedee8b88660cadde8b041077e73a79598b60f0e30bf60090a8baa5",
  "5dc2356eb4381c3d10341cbab83d767dd91ab527d7d394b9a5b4b7fc293b9803"
]

bitcoin-cli -regtest getblockcount
101
```

채굴자의 잔고를 확인해봅니다. 50 비트코인이 생성된 것을 확인할 수 있습니다.

```
bitcoin-cli -regtest getbalance
50.00000000
```

이제 이 비트코인을 송금 받을 거래에 사용할 계좌를 생성하고 잔고를 확인해봅니다.

```
bitcoin-cli -regtest getnewaddress comafire
2Mzy66ig8SwcC7UdQVrzYHtRDxZgBoxdq3G

bitcoin-cli -regtest getbalance comafire
0.00000000
```

채굴자에게서 비트코인을 받아봅니다. 송금시에는 계좌 주소가 필요합니다. 송금을 하면 송장번호라 할수 있는 트랜잭션 인덱스가 출력되고 이를 확인해 볼수 있습니다. 아직 송금이 완료되지는 않은 상태라 미완료 인덱스를 볼수 있도록 listunspent 명령에 0 값을 넣어 확인합니다.

```
bitcoin-cli -regtest sendtoaddress 2Mzy66ig8SwcC7UdQVrzYHtRDxZgBoxdq3G 5
ddc39e2c235a0357d27005602ecc83143b01caf51976c7c2a4a3abcf474744f7

bitcoin-cli -regtest listunspent 0
[
  {
    "txid": "ddc39e2c235a0357d27005602ecc83143b01caf51976c7c2a4a3abcf474744f7",
    "vout": 0,
    "address": "2Mzy66ig8SwcC7UdQVrzYHtRDxZgBoxdq3G",
    "account": "comafire",
    "redeemScript": "0014da2f417e870b024d832839a790282a9185e40f4d",
    "scriptPubKey": "a91454b3c3f3c71578881eaaa0b540ad7710babd917287",
    "amount": 5.00000000,
    "confirmations": 0,
    "spendable": true,
    "solvable": true,
    "safe": true
  },
  {
    "txid": "ddc39e2c235a0357d27005602ecc83143b01caf51976c7c2a4a3abcf474744f7",
    "vout": 1,
    "address": "2MwhGAo4Y5BBEPnt9mJBpFjNZRDGBRfYN7x",
    "redeemScript": "0014a88a6ade15594dc14b92fb985398378d79bace28",
    "scriptPubKey": "a91430cd23915145b69463dde98b080f43b10e2e37ea87",
    "amount": 44.99996240,
    "confirmations": 0,
    "spendable": true,
    "solvable": true,
    "safe": true
  }
]
```

송금은 완료 되지 않았지만, 수수료는 이미 떼어져 있네요.

이제 채굴을 통해서 미완료 트랙잰션을 완료하고 계좌 잔고를 확인해봅니다.

```
bitcoin-cli -regtest generate 1
[
  "623f16c369a64e3b14da59aa1a901d9b28b6d25d875219ae62d714acb266894f"
]

bitcoin-cli -regtest listunspent

bitcoin-cli -regtest getbalance comafire
5.00000000
```

5 비트코인이 입금된 것을 확인해 볼수 있습니다.

이제 기록상에서도 송금된 내역을 확인해 볼 수 있습니다.

```
bitcoin-cli -regtest listunspent
[
  {
    "txid": "fbd7979b39d018d235fc61cc9d895b6bed6aa385fc15db11a5f006f18d59d1ef",
    "vout": 0,
    "address": "miV8mJqKPBtRSVdyhKPMpy6CQsy7omodJK",
    "scriptPubKey": "2102cf0d8f1c775ef3697128eea2c8a18082f10498306cd9d032f96c3f50736bfa29ac",
    "amount": 50.00000000,
    "confirmations": 101,
    "spendable": true,
    "solvable": true,
    "safe": true
  },
  {
    "txid": "ddc39e2c235a0357d27005602ecc83143b01caf51976c7c2a4a3abcf474744f7",
    "vout": 0,
    "address": "2Mzy66ig8SwcC7UdQVrzYHtRDxZgBoxdq3G",
    "account": "comafire",
    "redeemScript": "0014da2f417e870b024d832839a790282a9185e40f4d",
    "scriptPubKey": "a91454b3c3f3c71578881eaaa0b540ad7710babd917287",
    "amount": 5.00000000,
    "confirmations": 1,
    "spendable": true,
    "solvable": true,
    "safe": true
  },
  {
    "txid": "ddc39e2c235a0357d27005602ecc83143b01caf51976c7c2a4a3abcf474744f7",
    "vout": 1,
    "address": "2MwhGAo4Y5BBEPnt9mJBpFjNZRDGBRfYN7x",
    "redeemScript": "0014a88a6ade15594dc14b92fb985398378d79bace28",
    "scriptPubKey": "a91430cd23915145b69463dde98b080f43b10e2e37ea87",
    "amount": 44.99996240,
    "confirmations": 1,
    "spendable": true,
    "solvable": true,
    "safe": true
  }
]
```

